### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import librosa
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from algorithms import encode_columns, svm_model, accuracy_calculator, random_forest_model, array_column_spread
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

### Loading the dataset using Pandas
The data is found in development.csv (development set): a comma-separated values file containing the records from the development set. This portion does have the action and object columns, which you should use to obtain the labels to train and validate your models.
The dataset consists in a collection of audio file in a WAV format. 

Each record is characterized by several attributes. The following is a short description for each of them.
- path: the path of the audio file.
- speakerId: the id of the speaker.
- action: the type of action required through the intent.
- object: the device involved by intent.
- Self-reported fluency level: the speaking fluency of the speaker.
- First Language spoken: the first language spoken by the speaker.
- Current language used for work/school: the main language spoken by the speaker during daily activities.
- gender: the gender of the speaker.
- ageRange: the age range of the speaker.

In [2]:
df = pd.read_csv("dsl_data/development.csv")
df_eval = pd.read_csv("dsl_data/evaluation.csv")

In [3]:
y = df.values[:,3:5].sum(axis=1)
y

array(['change languagenone', 'activatemusic', 'deactivatelights', ...,
       'deactivatelights', 'deactivatelights', 'increasevolume'],
      dtype=object)

In [4]:
column_names = ["Self-reported fluency level ","First Language spoken", "Current language used for work/school", "speakerId", "gender","ageRange"]


In [5]:
encode_columns(df_eval, column_names)

In [6]:

encode_columns(df,column_names)

In [7]:
def audio_feature_extraction(df):
        data_array= []
        rate_array = []
        for audio in df['path']:
                data, rate = librosa.load(audio)
                data_array.append(data)
                rate_array.append(rate)

        df['data'] = data_array
        df['rate'] = rate_array

In [8]:
audio_feature_extraction(df)
audio_feature_extraction(df_eval)

In [9]:
from scipy.stats import skew, kurtosis

def time_domain(df, column):
    df[f'{column}_mean'] = df[column].apply(lambda x: np.mean(x))
    df[f'{column}_min'] = df[column].apply(lambda x: np.min(x))
    df[f'{column}_max'] = df[column].apply(lambda x: np.max(x))
    df[f'{column}_skew'] = df[column].apply(lambda x: skew(x))
    df[f'{column}_kurtosis'] = df[column].apply(lambda x: kurtosis(x))
    df[f'{column}_std'] = df[column].apply(lambda x: np.std(x))

### Chroma feature

In [10]:
def chroma_feature(df):
    chroma_array = []
    for data, rate in zip(df['data'], df['rate']):
        chroma = librosa.feature.chroma_stft(y=data, sr=rate)
        chroma_mean = np.mean(chroma, axis=1)
        chroma_array.append(chroma_mean)

    df['chroma'] = chroma_array

In [11]:
chroma_feature(df)
chroma_feature(df_eval)

In [12]:
time_domain(df, 'chroma')
time_domain(df_eval, 'chroma')

### Tonnetz feature

In [13]:
def tonnetz_feature(df):
    tonnetz_array = []
    for data, rate in zip(df['data'], df['rate']):
        tonnetz = librosa.feature.tonnetz(y=data, sr=rate)
        tonnetz_mean = np.mean(tonnetz, axis=1)
        tonnetz_array.append(tonnetz_mean)

    df['tonnetz'] = tonnetz_array

In [14]:
tonnetz_feature(df)
tonnetz_feature(df_eval)

/Users/hadiibrahim/Dev/POLITO/DSL-Project/NLP-Intent-detection/venv/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=641
  return f(*args, **kwargs)
/Users/hadiibrahim/Dev/POLITO/DSL-Project/NLP-Intent-detection/venv/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=960
  return f(*args, **kwargs)
/Users/hadiibrahim/Dev/POLITO/DSL-Project/NLP-Intent-detection/venv/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=480
  return f(*args, **kwargs)
/Users/hadiibrahim/Dev/POLITO/DSL-Project/NLP-Intent-detection/venv/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=673
  return f(*args, **kwargs)
/Users/hadiibrahim/Dev/POLITO/DSL-Project/NLP-Intent-detection/venv/lib/python3.10/site-packages/librosa

In [15]:
time_domain(df, 'tonnetz')
time_domain(df_eval, 'tonnetz')

### Spectral Contrast

In [64]:
def spectral_contrast(df):
    spectral_contrast_array = []
    for data, rate in zip(df['data'], df['rate']):
        spectral_contrast = librosa.feature.spectral_contrast(y=data, sr=rate)
        spectral_contrast_mean = np.mean(spectral_contrast, axis=1)
        spectral_contrast_array.append(spectral_contrast_mean)
    df['spectral_contrast'] = spectral_contrast_array    

In [65]:
spectral_contrast(df)
spectral_contrast(df_eval)


In [67]:
time_domain(df,'spectral_contrast')
time_domain(df_eval,'spectral_contrast')

### Root Mean Square Energy(RMSE)

In [106]:
def rmse_feature(df):
    # create an empty list to store the RMSE values
    rmse_list = []

    # iterate through the audio files in the dataset
    for data, rate in zip(df['data'], df['rate']):
        # calculate the root mean square energy
        rmse = librosa.feature.rms(y=data)
        # append the rmse mean to the rmse_list
        rmse_list.append(rmse[0])

    # add the rmse_list as a new column to the dataframe
    df['rmse'] = rmse_list

In [107]:
rmse_feature(df)
rmse_feature(df_eval)

In [109]:
time_domain(df, 'rmse')
time_domain(df_eval, 'rmse')

### Spectral Flatness (SF)

In [114]:
def spectral_flatness(df):
    # create an empty list to store the SF values
    sf_list = []

    # iterate through the audio files in the dataset
    for data, rate in zip(df['data'], df['rate']):
        # calculate the spectral flatness
        sf = librosa.feature.spectral_flatness(y=data)
        # append the SF mean to the sf_list
        sf_list.append(sf[0])

    # add the sf_list as a new column to the dataframe
    df['sf'] = sf_list

In [115]:
spectral_flatness(df)
spectral_flatness(df_eval)

In [116]:
df['sf'][0]

array([2.2572016e-02, 1.6183635e-02, 1.8982619e-02, 2.3756729e-02,
       1.7949011e-02, 1.9214276e-02, 2.4735810e-02, 2.6906725e-02,
       2.0292507e-02, 1.8287098e-02, 5.2938419e-03, 2.8689851e-03,
       3.9771921e-03, 1.4738076e-02, 5.1843724e-03, 1.8129633e-03,
       7.9851784e-04, 8.1938371e-04, 1.3511568e-04, 3.1879943e-05,
       2.9547515e-05, 3.7044385e-05, 3.3422755e-05, 2.6718566e-05,
       2.9119001e-05, 2.6837897e-05, 1.6286516e-05, 5.1540615e-06,
       1.5813862e-06, 8.4011162e-06, 1.1096295e-04, 6.5260916e-04,
       7.2922453e-04, 7.7021593e-04, 3.6829759e-04, 1.2930618e-04,
       5.4232158e-05, 4.0523159e-05, 5.2582574e-05, 3.3064745e-05,
       2.2726212e-05, 2.0018520e-05, 2.2983770e-05, 2.3635399e-05,
       3.7159443e-05, 5.1298444e-05, 6.4837637e-05, 3.7286420e-05,
       1.0426162e-05, 1.4207246e-05, 2.2579954e-05, 2.0837959e-05,
       2.6995043e-05, 3.1219755e-05, 4.8725404e-05, 4.2572228e-05,
       4.1085204e-05, 4.8519119e-05, 3.4952554e-05, 5.6051616e

In [117]:
time_domain(df,'sf')
time_domain(df_eval,'sf')

### Spectral Roll-off (SRO)

In [85]:
def sro_feature(df):
    # Create an empty list to store the spectral roll-off values
    spectral_rolloff_array = []

    for data, rate in zip(df['data'], df['rate']):
        spectral_rolloff = librosa.feature.spectral_rolloff(y=data, sr=rate)
        spectral_rolloff_array.append(spectral_rolloff[0])

    # Add the spectral roll-off values to the dataframe as a new column
    df['spectral_rolloff'] = spectral_rolloff_array

In [86]:
sro_feature(df)
sro_feature(df_eval)

In [87]:
df['spectral_rolloff'][0]

array([5361.76757812, 5340.234375  , 5340.234375  , 5523.26660156,
       5706.29882812, 5792.43164062, 5684.765625  , 5534.03320312,
       5361.76757812, 5329.46777344, 4791.13769531, 4575.80566406,
       4898.80371094, 6309.22851562, 6459.9609375 , 6578.39355469,
       6599.92675781, 6406.12792969, 5318.70117188, 4188.20800781,
       4371.24023438, 4317.40722656, 4328.17382812, 4209.74121094,
       4198.97460938, 3897.50976562, 3402.24609375, 2433.25195312,
        764.42871094, 2777.78320312, 5943.1640625 , 6276.92871094,
       6212.32910156, 5964.69726562, 5243.33496094, 2982.34863281,
       2982.34863281, 2982.34863281, 3003.88183594, 2939.28222656,
       2917.74902344, 2971.58203125, 3068.48144531, 3090.01464844,
       3488.37890625, 3779.07714844, 4080.54199219, 3520.67871094,
       1894.921875  , 1453.49121094, 1959.52148438, 2853.14941406,
       3036.18164062, 3090.01464844, 3477.61230469, 3832.91015625,
       3671.41113281, 3768.31054688, 3725.24414062, 4715.77148

In [88]:
time_domain(df,'spectral_rolloff')
time_domain(df_eval,'spectral_rolloff')

### Zero-Crossing Rate

In [28]:
def zcr_feature(df):
    # Create arrays to store the zero-crossing rate values
    zero_crossing_rate_array = []

    for data in df['data']:
        
        # Compute the zero-crossing rate for the current audio file
        zero_crossing_rate = sum(librosa.zero_crossings(data))
        prin
        # Append the zero-crossing rate to the zero_crossing_rate_array
        zero_crossing_rate_array.append(zero_crossing_rate)

    # Add the zero-crossing rate arrays as new columns in the dataframe
    df['zero_crossing_rate'] = zero_crossing_rate_array

In [29]:
zcr_feature(df)
zcr_feature(df_eval)

### Mel-frequency cepstral coefficients (MFCC)

In [31]:
def mfcc_feature(df):
    # Create arrays to store the mfcc rate values
    mfcc_array = []

    for data, rate in zip(df['data'], df['rate']):
        
        # Compute the mfccs for the current audio file
        mfcc = librosa.feature.mfcc(y=data, sr=rate, n_mfcc=50)
        # Compute the mean of the mfccs and append it to the mfcc_array
        mfcc_mean = np.mean(mfcc, axis=1)
        mfcc_array.append(mfcc_mean)

    # Add the mfcc as a new column in the dataframe
    df['mfcc'] = mfcc_array

In [32]:
mfcc_feature(df)
mfcc_feature(df_eval)

In [33]:
time_domain(df,'mfcc')
time_domain(df_eval,'mfcc')

In [34]:
df.columns

Index(['Id', 'path', 'action', 'object', 'Self-reported fluency level ',
       'First Language spoken', 'Current language used for work/school',
       'speakerId', 'gender', 'ageRange', 'data', 'rate', 'chroma',
       'chroma_mean', 'chroma_min', 'chroma_max', 'chroma_skew',
       'chroma_kurtosis', 'chroma_std', 'tonnetz', 'tonnetz_mean',
       'tonnetz_min', 'tonnetz_max', 'tonnetz_skew', 'tonnetz_kurtosis',
       'tonnetz_std', 'spectral_contrast', 'spectral_contrast_mean',
       'spectral_contrast_min', 'spectral_contrast_max',
       'spectral_contrast_skew', 'spectral_contrast_kurtosis',
       'spectral_contrast_std', 'rmse', 'rmse_mean', 'rmse_min', 'rmse_max',
       'rmse_skew', 'rmse_kurtosis', 'rmse_std', 'sf', 'sf_mean', 'sf_min',
       'sf_max', 'sf_skew', 'sf_kurtosis', 'sf_std', 'spectral_rolloff',
       'spectral_rolloff_mean', 'spectral_rolloff_min', 'spectral_rolloff_max',
       'spectral_rolloff_skew', 'spectral_rolloff_kurtosis',
       'spectral_rolloff_

In [38]:
features = ['Self-reported fluency level ',
       'First Language spoken',
       'speakerId', 'gender', 'ageRange',
       'chroma_mean', 'chroma_min', 'chroma_max', 'chroma_min',
       'chroma_max', 'chroma_skew', 'chroma_kurtosis', 'chroma_std',
       'tonnetz_mean', 'tonnetz_min', 'tonnetz_max', 'tonnetz_skew',
       'tonnetz_kurtosis', 'tonnetz_std',
       'spectral_contrast_mean', 'spectral_contrast_min',
       'spectral_contrast_max', 'spectral_contrast_skew',
       'spectral_contrast_kurtosis', 'spectral_contrast_std',
       'rmse_mean', 'rmse_min', 'rmse_max',
       'rmse_std', 'sf_mean', 'sf_min', 'sf_max', 'sf_std','spectral_rolloff_mean',
       'spectral_rolloff_min', 'spectral_rolloff_max','spectral_rolloff_std',
       'zero_crossing_rate_mean',
       'zero_crossing_rate_min', 'zero_crossing_rate_max',
       'zero_crossing_rate_std','mfcc_mean', 'mfcc_min', 'mfcc_max',
       'mfcc_skew', 'mfcc_kurtosis', 'mfcc_std']

In [39]:
len(features)

47

In [40]:
X_eval = df_eval[features].copy()
X_eval

X_eval.to_csv('x_eval.csv')

In [41]:
X = df[features].copy()
X
X.to_csv('x.csv')

In [43]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

selector = SelectKBest(score_func=chi2, k=30)
X_new = selector.fit_transform(X, y)

# Get the selected feature names
feature_names = X.columns[selector.get_support()]
print(feature_names)

ValueError: Input X must be non-negative.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y_pred_eval = svm_model(X, y, X_eval)

predictions = pd.DataFrame(y_pred_eval,columns=["Predicted"])
predictions.to_csv('my_data.csv', index=True)

In [ ]:
y_pred = svm_model(X_train, y_train, X_test)
svm_accuracy = accuracy_calculator(y_test, y_pred)

In [ ]:
svm_accuracy

In [ ]:
y_pred_eval = random_forest_model(X , X_eval, y)

predictions = pd.DataFrame(y_pred_eval,columns=["Predicted"])
predictions.to_csv('predictions.csv', index=True)


In [ ]:
y_pred = random_forest_model(X_train, X_test, y_train)
random_forest_accuracy = accuracy_calculator(y_test, y_pred)
random_forest_accuracy

In [ ]:
## Implementation using k-fold
from sklearn.model_selection import KFold, cross_val_score
# define the number of folds and whether to shuffle the data
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# create an instance of the SVM model
clf = SVC()

# use cross_val_score function to perform k-fold cross-validation
scores = cross_val_score(clf, X, y, cv=kf, scoring='accuracy')

# print the mean accuracy and standard deviation
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=0)

# shuffle the data before performing k-fold cross validation
X, y = shuffle(X, y)

# perform k-fold cross validation with 5 folds
scores = cross_val_score(clf, X, y, cv=5)

# calculate the mean accuracy of the model across all folds
accuracy = np.mean(scores)
print("Accuracy:", accuracy)
